# Stable Diffusion notebook by [@alexpopo](https://github.com/alexpopo/alex_stable_diffusion_webui)

This let's you generate images with CompVis/Stability [Stable Diffusion](https://github.com/CompVis/stable-diffusion) with bonus [KLMS sampling](https://github.com/crowsonkb/k-diffusion.git) from [@RiversHaveWings](https://twitter.com/RiversHaveWings)

You need to get the ckpt file and put it on your Google Drive first to use this. It can be downloaded from [HuggingFace](https://huggingface.co/runwayml/stable-diffusion-v1-5).

Other Stable Diffusion tools:
* [Deforum Stable Diffusion](https://colab.research.google.com/github/deforum/stable-diffusion/blob/main/Deforum_Stable_Diffusion.ipynb) by [deforum_art](https://twitter.com/deforum_art) team
* [HuggingFace space for Stable Diffusion](https://huggingface.co/spaces/stabilityai/stable-diffusion)
* [Stable Diffusion Interpolation](https://colab.research.google.com/drive/1EHZtFjQoRr-bns1It5mTcOVyZzZD9bBc?usp=sharing) by [@ygantigravity](https://twitter.com/ygantigravity)
* [Stable Diffusion with diffusers](https://colab.research.google.com/github/huggingface/notebooks/blob/main/diffusers/stable_diffusion.ipynb#scrollTo=zHkHsdtnry57)
* [Prompt-space Interpolation](https://colab.research.google.com/drive/1gj8_EPyntZuaiQuzYOzog5YKA74esoB3#scrollTo=3g5rSm43STsW) by [@DigThatData](https://twitter.com/DigThatData)
* [Stable Diffusion Lite](https://colab.research.google.com/drive/1cl3d84B5AXepycKAfCba77faUFBRGY2O?usp=sharing#scrollTo=Rn06QzsdsICR) by [@future__art](https://twitter.com/future__art)
* [Stable Diffusion web UI](https://github.com/AUTOMATIC1111/stable-diffusion-webui)
* More to come!!

If you're looking for more Ai art tools check here [Ai generative art tools list](https://pharmapsychotic.com/tools.html).

if you encounter any issues, feel free to discuss them.[Discord Support](https://discord.com/channels/1002292111942635562)

In [1]:
#@title Check GPU
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-ff3b3b69-6778-23f0-e3cf-2063722c6c4f)


In [2]:
#@title Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
#@title Check Environment
#@markdown ##### 谷歌云端硬盘：仅第一次需要下载模型文件，之后持久保存在谷歌云端硬盘
#@markdown ##### 简单便捷方式：每次重新连接CoLab需要重新下载所有模型文件，不需要云硬盘

import os
import time

if os.path.exists("/content/gdrive/MyDrive/"):
  print('Gdrive connected, using Gdrive storage ...')
  print("仅第一次需要下载模型文件，之后持久保存在谷歌云端硬盘")
else:
  print('[1;31mGdrive not connected, using colab storage ...')
  print("每次重新连接CoLab需要重新下载所有模型文件")

time.sleep(1)

outputs_path = "/content/SDWebUI"
%mkdir -p $outputs_path
print()
%cd $outputs_path

Gdrive connected, using Gdrive storage ...
仅第一次需要下载模型文件，之后持久保存在谷歌云端硬盘

/content/SDWebUI


In [4]:
#@title First Base Requirements

# 在尝试安装之前，请确保满足所有必需的依赖项
print("Check Common Software Versions ...")
!git --version
print()
print("git-v2.12及之后版本集成了git-lfs")
print("-"*80)
!apt --version
print("-"*80)
!python3 --version
print("-"*80)
!wget --version | grep "^GNU Wget"
print("-"*80)

print('Installing base requirements...')

# Debian-based:
!sudo apt install python3-venv

Check Common Software Versions ...
git version 2.25.1

git-v2.12及之后版本集成了git-lfs
--------------------------------------------------------------------------------
apt 2.0.9 (amd64)
--------------------------------------------------------------------------------
Python 3.8.10
--------------------------------------------------------------------------------
GNU Wget 1.20.3 built on linux-gnu.
--------------------------------------------------------------------------------
Installing base requirements...
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  python-pip-whl python3.8-venv
The following NEW packages will be installed:
  python-pip-whl python3-venv python3.8-venv
0 upgraded, 3 newly installed, 0 to remove and 27 not upgraded.
Need to get 1,812 kB

In [5]:
#@title Next Base Requirements

import os

# Clone WebUI Repo Code And Download SD Model
# Clone WebUI Repo Code
!git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git $outputs_path/stable-diffusion-webui

"""
Git LFS Details
SHA256: fe4efff1e174c627256e44ec2991ba279b3816e364b49f9be2abc0b3ff3f8556
Pointer size: 135 Bytes
Size of remote file: 4.27 GB
"""
# Download Stable Diffusion Model Checkpoint
# 下载模型文件"sd-v1-4.ckpt"并进行hash校验，检查是否下载完整
if os.path.exists("/content/gdrive/MyDrive/"):
  %mkdir -p /content/gdrive/MyDrive/AI/models/Stable-diffusion
  %cd /content/gdrive/MyDrive/AI/models/Stable-diffusion
  %rm -r /content/SDWebUI/stable-diffusion-webui/models/Stable-diffusion
  !ln -s /content/gdrive/MyDrive/AI/models/Stable-diffusion /content/SDWebUI/stable-diffusion-webui/models
  if not os.path.exists("sd-v1-4.ckpt"):
    !wget https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt -O /content/gdrive/MyDrive/AI/models/Stable-diffusion/sd-v1-4.ckpt
    !sha256sum sd-v1-4.ckpt
else:
  !wget https://huggingface.co/CompVis/stable-diffusion-v-1-4-original/resolve/main/sd-v1-4.ckpt -O $outputs_path/stable-diffusion-webui/models/Stable-diffusion/sd-v1-4.ckpt
  !sha256sum $outputs_path/stable-diffusion-webui/models/Stable-diffusion/sd-v1-4.ckpt

model_param = "/content/SDWebUI/stable-diffusion-webui/models/Stable-diffusion/sd-v1-4.ckpt"

Cloning into '/content/SDWebUI/stable-diffusion-webui'...
remote: Enumerating objects: 16079, done.
remote: Total 16079 (delta 0), reused 0 (delta 0), pack-reused 16079
Receiving objects: 100% (16079/16079), 27.23 MiB | 12.40 MiB/s, done.
Resolving deltas: 100% (11293/11293), done.
/content/gdrive/MyDrive/AI/models/Stable-diffusion


In [6]:
#@title Automatic Installation (Take a ten minute break with a cup of tea)

%cd $outputs_path/stable-diffusion-webui

!sed -i 's@#install_dir="/home/$(whoami)"@install_dir="/content/SDWebUI"@' $outputs_path/stable-diffusion-webui/webui-user.sh
!sed -i "s@can_run_as_root=0@can_run_as_root=1@" $outputs_path/stable-diffusion-webui/webui.sh

!bash webui.sh

/content/SDWebUI/stable-diffusion-webui

################################################################
Install script for stable-diffusion + Web UI
Tested on Debian 11 (Bullseye)
################################################################

################################################################
Running on root user
################################################################

################################################################
Repo already cloned, using it as install directory
################################################################

################################################################
Create and activate python venv
################################################################

################################################################
Launching launch.py...
################################################################
Python 3.8.10 (default, Nov 14 2022, 12:59:47) 
[GCC 9.4.0]
Commit hash: ea9bd9fc7409109adcd61b897abc2

In [9]:
# WebUI自动启动成功会有下面提示，停止上面的自动运行，添加安装'xformers'模块
# Running on local URL:  http://127.0.0.1:7860
# To create a public link, set `share=True` in `launch()`.

# Launching Web UI with arguments: No module 'xformers' Issues.
# fix bug: install xformers-0.0.16-cp38-cp38-manylinux2014_x86_64.whl
%cd /content/SDWebUI/stable-diffusion-webui

# !source tutorial-env/bin/activate; pip3 install --no-deps xformers==0.0.16rc425; pip3 list | grep -e "xformers"
!source venv/bin/activate; python3 -m pip install xformers==0.0.16; python3 -m pip list | grep -e "xformers"

/content/SDWebUI/stable-diffusion-webui
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
xformers                0.0.16         


In [14]:
#@title Restart Stable-Diffusion
import os

#@markdown Add credentials to your Gradio interface (optional).
User = "alexpopo" #@param {type:"string"}
Password= "123456" #@param {type:"string"}

auth=f"--gradio-auth {User}:{Password}"
if User =="" or Password=="":
  auth=""

%cd $outputs_path/stable-diffusion-webui

# Error while finding module specification for 'webui.py' 
# (ModuleNotFoundError: __path__ attribute not found on 'webui' while trying to find 'webui.py')
# !source venv/bin/activate; python3 -m webui.py --share --xformers

try:
  if os.path.isfile(model_param):
    !source venv/bin/activate; python3 webui.py --share --xformers $auth --theme="dark" --ckpt "$model_param"
  else:
    !source venv/bin/activate; python3 webui.py --share --xformers $auth --theme="light" --ckpt-dir "$model_param"
except:
  !source venv/bin/activate; python3 webui.py --share --xformers $auth

/content/SDWebUI/stable-diffusion-webui
Loading weights [fe4efff1e1] from /content/SDWebUI/stable-diffusion-webui/models/Stable-diffusion/sd-v1-4.ckpt
Creating model from config: /content/SDWebUI/stable-diffusion-webui/configs/v1-inference.yaml
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
Applying xformers cross attention optimization.
Textual inversion embeddings loaded(0): 
Model loaded in 29.8s (load weights from disk: 20.5s, create model: 1.4s, apply weights to model: 2.0s, apply half(): 1.2s, load VAE: 4.2s, move model to device: 0.6s).
Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://780a319d-1aba-4b96.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
Consuming a byte in the end state
Consuming a byte in the end state
  0% 0/50 [00:00<?, ?it/s]
  2% 1/50 [00:10<08:31, 10.43s/it]
  4% 2/50 [00:11<03:57,  4.94s/it]
 

# Prompt Demo

* green sapling rowing out of ground, mud, dirt, grass, high quality, photorealistic, sharp focus, depth of field

* Steps: 20, Sampler: Euler a, CFG scale: 12, Seed: 1441787169, Size: 512x512, Model hash: fe4efff1e1, Model: sd-v1-4